In [19]:
# %% Import necessary libraries
import matplotlib.pyplot as plt

import numpy as np

import adi

from comms_lib.pluto import Pluto

# Import existing functions
from imports import *
from digital_modulation import digital_modulation
from create_message import create_message
from encode_bits import encode_bits
from decode_message import decode_message
from decode_symbols import decode_symbols


In [28]:
!iio_info -s

Unable to create Local IIO context : Function not implemented (78)
No IIO context found.


In [30]:
sdr = Pluto("usb:0.5.5")

In [33]:
def transmit_image(sdr, image_path, M=2, K=5):
    
    from PIL import Image
    import io
    
    # Load and process image
    img = Image.open(image_path)
    img = img.resize((64, 64))
    img = img.convert('L')
    
    # Convert to bytes then bits
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='PNG')
    img_data = img_bytes.getvalue()
    
    image_bits = []
    for byte in img_data:
        for i in range(8):
            bit = (byte >> (7-i)) & 1
            image_bits.append(bit)
    
    print(f"Transmitting image: {len(image_bits)} bits ({len(img_data)} bytes)")
    
    # Configure SDR
    sdr.sample_rate = int(1e6)
    sdr.tx_lo = int(915e6)
    sdr.tx_hardwaregain_chan0 = 0
    
    # Modulate and transmit
    symbols = encode_bits(image_bits, M)
    message_signal = create_message(symbols, K)
    sdr.tx(message_signal)
    
    print("Transmission complete!")
    
    return len(image_bits), len(img_data), len(symbols), len(message_signal)

In [34]:
def receive_image(sdr, expected_bits, num_bytes, output_path, M=2, K=5):
    
    from PIL import Image
    import io
    
    print(f"Receiving image: {expected_bits} bits ({num_bytes} bytes)")
    
    # Configure SDR
    sdr.sample_rate = int(1e6)
    sdr.rx_lo = int(915e6)
    sdr.rx_hardwaregain_chan0 = 30
    
    # Calculate expected samples
    expected_symbols = expected_bits // int(np.log2(M))
    expected_samples = expected_symbols * K
    
    # Receive and demodulate
    rx_signal = sdr.rx()
    rx_signal = rx_signal[:expected_samples]
    symbols = decode_message(rx_signal, K)
    received_bits = decode_symbols(symbols, M)
    received_bits = received_bits[:expected_bits]
    
    # Convert bits back to image
    img_bytes = bytearray()
    for i in range(0, len(received_bits), 8):
        if i + 7 < len(received_bits):
            byte_val = 0
            for j in range(8):
                byte_val |= (received_bits[i+j] << (7-j))
            img_bytes.append(byte_val)
    
    img_bytes = img_bytes[:num_bytes]
    
    # Try to save image, handle corruption gracefully
    try:
        img = Image.open(io.BytesIO(img_bytes))
        img.save(output_path)
        print(f"Image saved to: {output_path}")
    except Exception as e:
        print(f"Image corrupted during transmission: {e}")
        # Save raw bytes for debugging
        with open("corrupted_data.bin", "wb") as f:
            f.write(img_bytes)
        print("Raw data saved to: corrupted_data.bin")
        
        # Create a simple test image to show structure
        try:
            test_img = Image.new('L', (64, 64), color=128)
            test_img.save("test_received.png")
            print("Created test image: test_received.png")
        except:
            pass
    
    return received_bits

In [39]:
def run_image_transmission(image_path, output_path="received_image.png", M=2, K=5):
    
    # Transmit image
    print("=== TRANSMISSION ===")
    num_bits, num_bytes, num_symbols, num_samples = transmit_image(sdr, image_path, M, K)

    import time
    time.sleep(0.5)
    
    # Receive image
    print("\n=== RECEPTION ===")
    received_bits = receive_image(sdr, num_bits, num_bytes, output_path, M, K)
    
    print(f"\n=== COMPLETE ===")
    print(f"Original: {image_path}")
    print(f"Received: {output_path}")
    
    return received_bits

In [40]:
run_image_transmission("grp9WT.png")

=== TRANSMISSION ===
Transmitting image: 4752 bits (594 bytes)
Transmission complete!

=== RECEPTION ===
Receiving image: 4752 bits (594 bytes)
Image corrupted during transmission: cannot identify image file <_io.BytesIO object at 0x12258c270>
Raw data saved to: corrupted_data.bin
Created test image: test_received.png

=== COMPLETE ===
Original: grp9WT.png
Received: received_image.png


[0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,


In [14]:
def run_image_transmission_debug(image_path, output_path="received_image.png", M=2, K=5):
    
    from PIL import Image
    import io
    
    print("=== TRANSMISSION ===")
    
    # Get original image bits for comparison
    img = Image.open(image_path)
    img = img.resize((64, 64))
    img = img.convert('L')
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='PNG')
    img_data = img_bytes.getvalue()
    
    original_bits = []
    for byte in img_data:
        for i in range(8):
            bit = (byte >> (7-i)) & 1
            original_bits.append(bit)
    
    # Transmit
    num_bits, num_bytes, num_symbols, num_samples = transmit_image(sdr, image_path, M, K)
    
    print("\n=== RECEPTION ===")
    received_bits = receive_image(sdr, num_bits, num_bytes, output_path, M, K)
    
    # Compare bits and count errors
    print(f"\n=== BIT ERROR ANALYSIS ===")
    bit_errors = sum(a != b for a, b in zip(original_bits[:len(received_bits)], received_bits))
    total_bits = len(received_bits)
    ber = bit_errors / total_bits if total_bits > 0 else 0
    
    print(f"Bit errors: {bit_errors}/{total_bits}")
    print(f"Bit Error Rate: {ber:.6f} ({ber*100:.3f}%)")
    
    if ber > 0.001:
        print("⚠️ High error rate - image corruption expected")
        print("Try increasing RX gain or reducing distance")
    elif ber > 0:
        print("⚠️ Some errors - image may be partially corrupted")
    else:
        print("✅ Perfect transmission!")
    
    return received_bits, bit_errors, ber

In [17]:
received_bits, errors, ber = run_image_transmission_debug("T_GL.jpg")

=== TRANSMISSION ===
Transmitting image: 13000 bits (1625 bytes)
Transmission complete!

=== RECEPTION ===
Receiving image: 13000 bits (1625 bytes)
Image corrupted during transmission: cannot identify image file <_io.BytesIO object at 0x114d3d210>
Raw data saved to: corrupted_data.bin
Created test image: test_received.png

=== BIT ERROR ANALYSIS ===
Bit errors: 6588/13000
Bit Error Rate: 0.506769 (50.677%)
⚠️ High error rate - image corruption expected
Try increasing RX gain or reducing distance


In [8]:
def debug_transmission():
    print("=== RF DEBUG ===")
    
    # Test with simple known pattern
    test_bits = [1, 0, 1, 0, 1, 0, 1, 0] * 10  # 80 bits alternating pattern
    
    print(f"Original test bits: {test_bits[:16]}...")
    
    # Configure SDR
    sdr.sample_rate = int(1e6)
    sdr.tx_lo = int(915e6) 
    sdr.rx_lo = int(915e6)
    sdr.tx_hardwaregain_chan0 = -10  # Increase TX power
    sdr.rx_hardwaregain_chan0 = 50   # Increase RX gain
    
    print(f"TX frequency: {sdr.tx_lo/1e6:.1f} MHz")
    print(f"RX frequency: {sdr.rx_lo/1e6:.1f} MHz")
    print(f"TX gain: {sdr.tx_hardwaregain_chan0} dB")
    print(f"RX gain: {sdr.rx_hardwaregain_chan0} dB")
    
    # Transmit test pattern
    symbols = encode_bits(test_bits, 2)
    message_signal = create_message(symbols, 5)
    sdr.tx(message_signal)
    print("Test pattern transmitted")
    
    # Receive 
    expected_symbols = len(test_bits)
    expected_samples = expected_symbols * 5
    
    rx_signal = sdr.rx()
    print(f"Received {len(rx_signal)} samples (expected {expected_samples})")
    
    # Check signal power
    signal_power = np.mean(np.abs(rx_signal)**2)
    print(f"Received signal power: {signal_power:.6f}")
    
    if signal_power < 1e-6:
        print("⚠️ Very low signal power - check antenna connections!")
    
    # Decode
    rx_signal = rx_signal[:expected_samples]
    symbols_rx = decode_message(rx_signal, 5)
    bits_rx = decode_symbols(symbols_rx, 2)
    bits_rx = bits_rx[:len(test_bits)]
    
    print(f"Received test bits: {bits_rx[:16]}...")
    
    # Compare
    errors = sum(a != b for a, b in zip(test_bits, bits_rx))
    ber = errors / len(test_bits)
    print(f"Test BER: {errors}/{len(test_bits)} = {ber:.3f} ({ber*100:.1f}%)")
    
    return test_bits, bits_rx, signal_power

# Run RF debug
debug_transmission()

=== RF DEBUG ===
Original test bits: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]...


/opt/anaconda3/envs/cosmos_final/lib/python3.13/site-packages/comms_lib/pluto.py:133: UserWarning: Receive gain is too high, clipping to 40 dB
  warnings.warn("Receive gain is too high, clipping to 40 dB")


TX frequency: 915.0 MHz
RX frequency: 915.0 MHz
TX gain: -30 dB
RX gain: 40 dB
Test pattern transmitted
Received 100000 samples (expected 400)
Received signal power: 362737.774520
Received test bits: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]...
Test BER: 0/80 = 0.000 (0.0%)


([1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0],
 [1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0],
 np.float64(362737.77452))

In [9]:
def run_image_transmission_debug(image_path, output_path="received_image.png", M=2, K=5):
    
    from PIL import Image
    import io
    
    print("=== TRANSMISSION ===")
    
    # Get original image bits for comparison
    img = Image.open(image_path)
    img = img.resize((64, 64))
    img = img.convert('L')
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='PNG')
    img_data = img_bytes.getvalue()
    
    original_bits = []
    for byte in img_data:
        for i in range(8):
            bit = (byte >> (7-i)) & 1
            original_bits.append(bit)
    
    # Transmit
    num_bits, num_bytes, num_symbols, num_samples = transmit_image(sdr, image_path, M, K)
    
    print("\n=== RECEPTION ===")
    received_bits = receive_image(sdr, num_bits, num_bytes, output_path, M, K)
    
    # Compare bits and count errors
    print(f"\n=== BIT ERROR ANALYSIS ===")
    bit_errors = sum(a != b for a, b in zip(original_bits[:len(received_bits)], received_bits))
    total_bits = len(received_bits)
    ber = bit_errors / total_bits if total_bits > 0 else 0
    
    print(f"Bit errors: {bit_errors}/{total_bits}")
    print(f"Bit Error Rate: {ber:.6f} ({ber*100:.3f}%)")
    
    if ber > 0.001:
        print("⚠️ High error rate - image corruption expected")
        print("Try increasing RX gain or reducing distance")
    elif ber > 0:
        print("⚠️ Some errors - image may be partially corrupted")
    else:
        print("✅ Perfect transmission!")
    
    return received_bits, bit_errors, ber

In [11]:
def transmit_raw_image(sdr, image_path, M=2, K=5):
    
    from PIL import Image
    
    print(f"Loading image: {image_path}")
    
    # Load and convert to simple grayscale pixels (no PNG compression)
    img = Image.open(image_path)
    img = img.resize((32, 32))  # Even smaller for reliability
    img = img.convert('L')
    
    # Get raw pixel values (0-255)
    pixels = list(img.getdata())
    
    # Convert pixels to bits (8 bits per pixel)
    image_bits = []
    for pixel in pixels:
        for i in range(8):
            bit = (pixel >> (7-i)) & 1
            image_bits.append(bit)
    
    print(f"Transmitting raw image: {len(image_bits)} bits ({len(pixels)} pixels)")
    
    # Configure and transmit
    sdr.sample_rate = int(1e6)
    sdr.tx_lo = int(915e6)
    sdr.tx_hardwaregain_chan0 = 0
    
    symbols = encode_bits(image_bits, M)
    message_signal = create_message(symbols, K)
    sdr.tx(message_signal)
    
    print("Raw image transmission complete!")
    
    return len(image_bits), len(pixels)

def receive_raw_image(sdr, expected_bits, num_pixels, output_path, M=2, K=5):
    
    from PIL import Image
    
    print(f"Receiving raw image: {expected_bits} bits ({num_pixels} pixels)")
    
    # Configure and receive
    sdr.sample_rate = int(1e6)
    sdr.rx_lo = int(915e6)
    sdr.rx_hardwaregain_chan0 = 40
    
    expected_symbols = expected_bits // int(np.log2(M))
    expected_samples = expected_symbols * K
    
    rx_signal = sdr.rx()
    rx_signal = rx_signal[:expected_samples]
    symbols = decode_message(rx_signal, K)
    received_bits = decode_symbols(symbols, M)
    received_bits = received_bits[:expected_bits]
    
    # Convert bits back to pixels
    pixels = []
    for i in range(0, len(received_bits), 8):
        if i + 7 < len(received_bits):
            pixel_val = 0
            for j in range(8):
                pixel_val |= (received_bits[i+j] << (7-j))
            pixels.append(pixel_val)
    
    # Create image from pixels
    pixels = pixels[:num_pixels]  # Ensure correct length
    img_size = int(np.sqrt(num_pixels))  # Assume square image
    img = Image.new('L', (img_size, img_size))
    img.putdata(pixels)
    img.save(output_path)
    
    print(f"Raw image saved to: {output_path}")
    
    return received_bits

# Test raw image transmission
def run_raw_image_transmission(image_path, output_path="received_raw.png"):
    
    print("=== RAW IMAGE TRANSMISSION ===")
    num_bits, num_pixels = transmit_raw_image(sdr, image_path, M=2, K=5)
    
    print("\n=== RAW IMAGE RECEPTION ===")
    received_bits = receive_raw_image(sdr, num_bits, num_pixels, output_path, M=2, K=5)
    
    print(f"\n=== COMPLETE ===")
    print(f"Original: {image_path}")
    print(f"Received: {output_path}")
    
    return received_bits

In [12]:
run_raw_image_transmission("grp9WT.png")

=== RAW IMAGE TRANSMISSION ===
Loading image: grp9WT.png
Transmitting raw image: 8192 bits (1024 pixels)
Raw image transmission complete!

=== RAW IMAGE RECEPTION ===
Receiving raw image: 8192 bits (1024 pixels)
Raw image saved to: received_raw.png

=== COMPLETE ===
Original: grp9WT.png
Received: received_raw.png


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [15]:
def debug_image_transmission():
    
    # Test with progressively longer bit sequences
    test_lengths = [80, 400, 1600, 8192]  # Build up to full image size
    
    for length in test_lengths:
        print(f"\n=== Testing {length} bits ===")
        
        # Create predictable test pattern
        test_bits = [i % 2 for i in range(length)]  # Alternating 0,1,0,1...
        
        # Configure SDR
        sdr.sample_rate = int(1e6)
        sdr.tx_lo = int(915e6)
        sdr.rx_lo = int(915e6)
        sdr.tx_hardwaregain_chan0 = -10
        sdr.rx_hardwaregain_chan0 = 40
        
        # Transmit
        symbols = encode_bits(test_bits, 2)
        message_signal = create_message(symbols, 5)
        sdr.tx(message_signal)
        
        # Receive
        expected_symbols = len(test_bits)
        expected_samples = expected_symbols * 5
        
        rx_signal = sdr.rx()
        rx_signal = rx_signal[:expected_samples]
        symbols_rx = decode_message(rx_signal, 5)
        bits_rx = decode_symbols(symbols_rx, 2)
        bits_rx = bits_rx[:len(test_bits)]
        
        # Check errors
        errors = sum(a != b for a, b in zip(test_bits, bits_rx))
        ber = errors / len(test_bits)
        
        print(f"Length: {length}, Errors: {errors}, BER: {ber:.6f} ({ber*100:.3f}%)")
        
        if ber > 0.01:  # More than 1% errors
            print(f"❌ Failed at {length} bits")
            break
        else:
            print(f"✅ Passed {length} bits")

# Run the debug
debug_image_transmission()


=== Testing 80 bits ===
Length: 80, Errors: 80, BER: 1.000000 (100.000%)
❌ Failed at 80 bits


In [16]:
def debug_exact_pattern():
    
    print("=== Testing with EXACT working pattern ===")
    
    # Use the exact same pattern that worked before
    test_bits = [1, 0, 1, 0, 1, 0, 1, 0] * 10  # 80 bits, starts with 1
    
    print(f"Pattern: {test_bits[:16]}...")
    
    # Configure SDR (same as before)
    sdr.sample_rate = int(1e6)
    sdr.tx_lo = int(915e6)
    sdr.rx_lo = int(915e6) 
    sdr.tx_hardwaregain_chan0 = -10
    sdr.rx_hardwaregain_chan0 = 40
    
    # Transmit
    symbols = encode_bits(test_bits, 2)
    message_signal = create_message(symbols, 5)
    sdr.tx(message_signal)
    
    # Receive
    expected_symbols = len(test_bits)
    expected_samples = expected_symbols * 5
    
    rx_signal = sdr.rx()
    rx_signal = rx_signal[:expected_samples]
    symbols_rx = decode_message(rx_signal, 5)
    bits_rx = decode_symbols(symbols_rx, 2)
    bits_rx = bits_rx[:len(test_bits)]
    
    print(f"Received: {bits_rx[:16]}...")
    
    # Check errors
    errors = sum(a != b for a, b in zip(test_bits, bits_rx))
    ber = errors / len(test_bits)
    
    print(f"Errors: {errors}/{len(test_bits)}, BER: {ber:.6f} ({ber*100:.3f}%)")
    
    return ber == 0

# Test the exact working pattern
debug_exact_pattern()

=== Testing with EXACT working pattern ===
Pattern: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]...
Received: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]...
Errors: 0/80, BER: 0.000000 (0.000%)


True

In [17]:
def debug_timing_issue():
    
    patterns_to_test = [
        [1, 0, 1, 0, 1, 0, 1, 0] * 10,  # Starts with 1 - should work
        [0, 1, 0, 1, 0, 1, 0, 1] * 10,  # Starts with 0 - currently fails
        [1, 1, 0, 0, 1, 1, 0, 0] * 20,  # Different pattern
        [0, 0, 1, 1, 0, 0, 1, 1] * 20,  # Different pattern starting with 0
    ]
    
    for i, test_bits in enumerate(patterns_to_test):
        print(f"\n=== Pattern {i+1}: {test_bits[:8]} ===")
        
        # Configure SDR
        sdr.sample_rate = int(1e6)
        sdr.tx_lo = int(915e6)
        sdr.rx_lo = int(915e6)
        sdr.tx_hardwaregain_chan0 = -10
        sdr.rx_hardwaregain_chan0 = 40
        
        # Transmit
        symbols = encode_bits(test_bits, 2)
        message_signal = create_message(symbols, 5)
        sdr.tx(message_signal)
        
        # Receive
        rx_signal = sdr.rx()
        rx_signal = rx_signal[:len(test_bits)*5]
        symbols_rx = decode_message(rx_signal, 5)
        bits_rx = decode_symbols(symbols_rx, 2)
        bits_rx = bits_rx[:len(test_bits)]
        
        # Check errors
        errors = sum(a != b for a, b in zip(test_bits, bits_rx))
        ber = errors / len(test_bits)
        
        print(f"Original:  {test_bits[:16]}")
        print(f"Received:  {bits_rx[:16]}")
        print(f"BER: {ber:.6f} ({ber*100:.1f}%)")

debug_timing_issue()


=== Pattern 1: [1, 0, 1, 0, 1, 0, 1, 0] ===
Original:  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
Received:  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
BER: 0.000000 (0.0%)

=== Pattern 2: [0, 1, 0, 1, 0, 1, 0, 1] ===
Original:  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
Received:  [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
BER: 1.000000 (100.0%)

=== Pattern 3: [1, 1, 0, 0, 1, 1, 0, 0] ===
Original:  [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]
Received:  [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]
BER: 0.500000 (50.0%)

=== Pattern 4: [0, 0, 1, 1, 0, 0, 1, 1] ===
Original:  [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1]
Received:  [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0]
BER: 1.000000 (100.0%)


In [18]:
def transmit_with_sync(sdr, data_bits, M=2, K=5):
    
    # Add known sync pattern at start
    sync_pattern = [1, 0, 1, 0, 1, 0, 1, 0] * 4  # 32 bits of working pattern
    full_bits = sync_pattern + data_bits
    
    print(f"Transmitting with sync: {len(full_bits)} bits ({len(sync_pattern)} sync + {len(data_bits)} data)")
    
    # Configure and transmit
    sdr.sample_rate = int(1e6)
    sdr.tx_lo = int(915e6)
    sdr.tx_hardwaregain_chan0 = 0
    
    symbols = encode_bits(full_bits, M)
    message_signal = create_message(symbols, K)
    sdr.tx(message_signal)
    
    return len(full_bits), len(sync_pattern), len(data_bits)

def receive_with_sync(sdr, total_bits, sync_bits, data_bits, M=2, K=5):
    
    # Configure and receive
    expected_samples = total_bits * K
    
    sdr.sample_rate = int(1e6)
    sdr.rx_lo = int(915e6)
    sdr.rx_hardwaregain_chan0 = 40
    
    rx_signal = sdr.rx()
    rx_signal = rx_signal[:expected_samples]
    symbols_rx = decode_message(rx_signal, K)
    bits_rx = decode_symbols(symbols_rx, 2)
    
    # Extract just the data part (skip sync)
    data_only = bits_rx[sync_bits:sync_bits + data_bits]
    
    print(f"Received {len(data_only)} data bits (skipped {sync_bits} sync bits)")
    
    return data_only

# Test with sync
test_data = [0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0] * 10  # 120 bits of test data
total_bits, sync_bits, data_bits = transmit_with_sync(sdr, test_data)
received_data = receive_with_sync(sdr, total_bits, sync_bits, data_bits)

errors = sum(a != b for a, b in zip(test_data, received_data))
print(f"Data errors: {errors}/{len(test_data)} = {errors/len(test_data):.1%}")

Transmitting with sync: 152 bits (32 sync + 120 data)
Received 120 data bits (skipped 32 sync bits)
Data errors: 57/120 = 47.5%


In [ ]:
# Transmit Function
def transmit_bits(sdr, data_bits, M=2, K=5):
    
    print(f"Transmitting {len(data_bits)} bits...")
    
    # Step 1: Convert bits to symbols
    symbols = encode_bits(data_bits, M)
    print(f"Generated {len(symbols)} symbols")
    
    # Step 2: Convert symbols to continuous signal
    message_signal = create_message(symbols, K)
    print(f"Created message signal: {len(message_signal)} samples")
    
    # Step 3: Transmit through SDR
    sdr.tx(message_signal)
    print("Transmission complete!")
    
    return len(symbols), len(message_signal)

# Receive Function
def receive_bits(sdr, expected_bits, M=2, K=5):
    
    print(f"Receiving {expected_bits} bits...")
    
    # Calculate expected signal length
    expected_symbols = expected_bits // int(np.log2(M))
    expected_samples = expected_symbols * K
    print(f"Expecting {expected_samples} samples ({expected_symbols} symbols)")
    
    # Step 1: Receive signal from SDR
    rx_signal = sdr.rx()
    print(f"Received {len(rx_signal)} samples")
    
    # Take only expected length
    rx_signal = rx_signal[:expected_samples]
    
    # Step 2: Convert signal to symbols
    symbols = decode_message(rx_signal, K)
    print(f"Decoded {len(symbols)} symbols")
    
    # Step 3: Convert symbols to bits
    data_bits = decode_symbols(symbols, M)
    print(f"Decoded {len(data_bits)} bits")
    
    # Take only expected number of bits
    data_bits = data_bits[:expected_bits]
    print("Reception complete!")
    
    return data_bits

# Setup SDR Connection
def setup_sdr(usb_address="usb:1.5.5"):
    
    sdr = Pluto(usb_address)
    
    # Configure SDR
    sdr.sample_rate = int(1e6)
    sdr.carrier_frequency = int(915e6)
    sdr.tx_gain = 0
    sdr.rx_gain = 30
    
    print(f"Connected to PlutoSDR at {usb_address}")
    print(f"Sample rate: {sdr.sample_rate/1e6:.1f} MHz")
    print(f"Carrier frequency: {sdr.carrier_frequency/1e6:.1f} MHz")
    
    return sdr

# Test Transmission
# Connect to SDR
sdr = setup_sdr()

# Test data
test_bits = [1, 0, 1, 1, 0, 0, 1, 0]
print(f"Original bits: {test_bits}")

# Transmit
num_symbols, num_samples = transmit_bits(sdr, test_bits, M=2, K=5)

# Test Reception
# Receive (run this after transmission)
received_bits = receive_bits(sdr, expected_bits=8, M=2, K=5)

print(f"Original bits:  {test_bits}")
print(f"Received bits:  {received_bits}")

# Check if transmission was successful
if test_bits == received_bits:
    print("✅ SUCCESS: Perfect transmission!")
else:
    print("❌ ERROR: Transmission failed")
    errors = sum(a != b for a, b in zip(test_bits, received_bits))
    print(f"Bit errors: {errors}/{len(test_bits)}")

ERROR: Unable to claim interface 0:5:5: Permission denied (13)


Exception: No device found

In [ ]:
# Image Transmission Functions
import PIL.Image as Image
import io

def image_to_bits(image_path):
    """Convert image file to bits for transmission"""
    
    # Load and resize image to manageable size
    img = Image.open(image_path)
    img = img.resize((64, 64))  # Small size for testing
    img = img.convert('L')  # Grayscale
    
    # Convert to bytes
    img_bytes = io.BytesIO()
    img.save(img_bytes, format='PNG')
    img_data = img_bytes.getvalue()
    
    # Convert bytes to bits
    bits = []
    for byte in img_data:
        for i in range(8):
            bits.append((byte >> (7-i)) & 1)
    
    return bits, len(img_data)

def bits_to_image(bits, num_bytes, output_path):
    """Convert received bits back to image file"""
    
    # Convert bits back to bytes
    img_bytes = bytearray()
    for i in range(0, len(bits), 8):
        if i + 7 < len(bits):
            byte_val = 0
            for j in range(8):
                byte_val |= (bits[i+j] << (7-j))
            img_bytes.append(byte_val)
    
    # Truncate to expected length
    img_bytes = img_bytes[:num_bytes]
    
    # Save as image
    img = Image.open(io.BytesIO(img_bytes))
    img.save(output_path)
    
    return output_path

def transmit_image(sdr, image_path, M=2, K=5):
    """Transmit image through SDR"""
    
    print(f"Loading image: {image_path}")
    image_bits, num_bytes = image_to_bits(image_path)
    print(f"Image converted to {len(image_bits)} bits ({num_bytes} bytes)")
    
    # Transmit the bits
    num_symbols, num_samples = transmit_bits(sdr, image_bits, M, K)
    
    print("Image transmission complete!")
    return len(image_bits), num_bytes

def receive_image(sdr, expected_bits, num_bytes, output_path, M=2, K=5):
    """Receive image through SDR"""
    
    print(f"Receiving image...")
    
    # Receive the bits
    received_bits = receive_bits(sdr, expected_bits, M, K)
    
    # Convert back to image
    print(f"Converting bits back to image: {output_path}")
    bits_to_image(received_bits, num_bytes, output_path)
    print("Image reconstruction complete!")
    
    return received_bits

# Image Transmission Test
# Connect to SDR
sdr = setup_sdr()

# Transmit image (put your image file here)
image_path = "T_GL.jpg"  # Change to your image file
num_bits, num_bytes = transmit_image(sdr, image_path, M=2, K=5)

print(f"\nTransmitted: {num_bits} bits, {num_bytes} bytes")
print(f"Tell receiver: expected_bits={num_bits}, num_bytes={num_bytes}")

# Receive image (run after transmission)
output_path = "received_image.png"
received_bits = receive_image(sdr, num_bits, num_bytes, output_path, M=2, K=5)

print(f"Image saved to: {output_path}")
print("Compare original and received images!")